In [1]:
%matplotlib inline
import os
import os.path
import csv
import numpy
import scipy.stats
import scipy.cluster
import itertools
import math
import sys
import sklearn.cluster
import sklearn.decomposition
import sklearn.manifold
import skimage.filters
import pandas
from matplotlib import pylab, mlab
import matplotlib.cm
import matplotlib.patches
import IPython.display
# local modules (in same dir as this notebook)
import plotutils
# import filterutils
import syspathutils


In [2]:
base_path = os.path.realpath(os.path.join(os.path.curdir, '..'))
data_path = os.path.join(base_path, 'samples')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/.env/lib/python2.7/site-packages/')
syspathutils.append_to_sys_path('/home/gio/langdev/gitclones/arbimon2-jobs/lib/')

In [3]:
datasets = {x.replace('sample', '').strip('_'):os.path.join(data_path, x) for x in os.listdir(data_path)}
print '\n'.join("{} : {}".format(x, y) for x, y in datasets.items())

sabana_seca : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/sabana_seca
el_verde : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/el_verde
validation_data : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/validation_data
amarakaeri : /home/gio/langdev/gitclones/mine/fltr-audio-seg/samples/amarakaeri


In [4]:
# dataset = 'smithsonian'
# dataset = 'el_verde1'
# dataset = 'sabana_seca1'
dataset = 'validation_data'
# roi_filepath = ('/home/gio/langdev/gitclones/arbimon2-jobs/synth_output/rois.txt')

roi_filepath = os.path.join(datasets[dataset], 'rois.txt')

rois_data_ori = pandas.read_csv(roi_filepath)
# compute tod and site name from recording filename
rois_data_ori['tod'] = rois_data_ori['rec'].apply(lambda x: int(x.split('.')[0].split('_')[-1].split('-')[0]) if 'undefined' not in x else 0)
# rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: int(x.split('/')[0]) if 'undefined' not in x else 0)
rois_data_ori['site'] = rois_data_ori['rec'].apply(lambda x: 1)

print "dataset:{}, sites:{}, recs:{}, rois:{}, rois per rec:{}".format(
    dataset,
    len(rois_data_ori.groupby('site')), 
    len(rois_data_ori.groupby('rec')), 
    len(rois_data_ori), 
    len(rois_data_ori) * 1.0 / len(rois_data_ori.groupby('rec'))
)

dataset:validation_data, sites:1, recs:6, rois:2167, rois per rec:361.166666667


In [5]:
import collections

recs = pandas.read_csv(os.path.join(datasets[dataset], 'recs.txt'))
vals = pandas.read_csv(os.path.join(datasets[dataset], 'validations.txt'), index_col=0)

def intersects(x0, x1, y0, y1):
    return not(x1 < y0 or y1 < x0)

def linear_intersection_length(l1x1, l1x2, l2x1, l2x2):
    return max(min(l1x2, l2x2) - max(l1x1, l2x1), 0)

def rectangle_area(x1, x2, y1, y2):
    return max(x2 - x1, 0) * max(y2 - y1, 0)

def intersection_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2):
    return linear_intersection_length(r1x1, r1x2, r2x1, r2x2) * linear_intersection_length(r1y1, r1y2, r2y1, r2y2)

def union_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2):
    return rectangle_area(r1x1, r1x2, r1y1, r1y2) + rectangle_area(r2x1, r2x2, r2y1, r2y2) - intersection_area(r1x1, r1x2, r1y1, r1y2, r2x1, r2x2, r2y1, r2y2)


# cov_roivals, cov_valrois = roivals, valrois
roivals, valrois = collections.defaultdict(list), collections.defaultdict(list)
cov_roivals, cov_valrois = collections.defaultdict(list), collections.defaultdict(list)

for i, roi in rois_data_ori.iterrows():
    for j, val in vals.iterrows():
        area_of_union = union_area(roi.t0, roi.t1, roi.f0, roi.f1, val.t0, val.t1, val.f0, val.f1)
        area_of_intersection = intersection_area(roi.t0, roi.t1, roi.f0, roi.f1, val.t0, val.t1, val.f0, val.f1)
        val_roi_coverage = area_of_intersection / area_of_union
        if val_roi_coverage > 0:
            cov_roivals[i].append([val_roi_coverage, j])
            cov_valrois[j].append([val_roi_coverage, i])
            
        if intersects(roi.t0, roi.t1, val.t0, val.t1) and intersects(roi.f0, roi.f1, val.f0, val.f1):
           roivals[i].append(j)
           valrois[j].append(i)

print len(roivals)
# len(vals) * len(rois_data_ori)
# del vals['idx']
# print vals.groupby('recording').groups.keys()
# vals.to_csv(os.path.join(datasets[dataset], 'validations.txt'))
# vals

2009


In [34]:
for i, valroi in valrois.items():
    cov_valroi_rois = [x[1] for x in cov_valrois[i]]
    missing = [x for x in valroi if x not in cov_valroi_rois]
    if missing and len(cov_valroi_rois) == 0:
        print i, missing, len(valroi), len(cov_valroi_rois)

2183 [1965] 1 0


In [67]:
valrois

defaultdict(<type 'list'>, {0: [[0.029251900829694037, 0], [1.0, 62], [0.09573559591343953, 1541], [0.03039564491257706, 1609], [0.07100554914733329, 1623], [0.012562247979922501, 1752]], 1: [[0.02000815395953821, 4], [1.0, 63], [0.0516051710450556, 1553], [0.05784573622010923, 1619], [0.03860336201652347, 1634], [0.07399249701215176, 1644], [0.053184469750822486, 1895]], 2: [[1.0, 64], [0.2976072619533287, 1627], [0.03199653952957991, 1643], [0.07409811905071008, 1739]], 3: [[1.0, 65], [0.12551749097380183, 1882]], 4: [[1.0, 66], [0.18964060240753597, 1720], [0.23081477289267968, 1908]], 5: [[1.0, 67], [0.1764644361292012, 1788]], 6: [[1.0, 68], [0.17054898455554404, 1810]], 7: [[1.0, 69]], 8: [[1.0, 70]], 9: [[1.0, 71]], 10: [[1.0, 72], [0.02310358111031486, 257], [0.33104722860856517, 1816]], 11: [[1.0, 73], [0.14959931796178402, 1817]], 12: [[1.0, 74]], 13: [[1.0, 75]], 14: [[1.0, 76], [0.057360324397085195, 1720]], 15: [[1.0, 77], [0.05977718427147891, 1363], [0.07462330689173419,

In [70]:
valroi_max_overlap_percentage = [max(y[0] for y in x) for x in valrois.values()]
{
    'total':  len(valrois),
    'det_val': len(filter(lambda x: x >= .25, valroi_max_overlap_percentage))
}

{'det_val': 1879, 'total': 2187}